In [1]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier

In [2]:
mydf= pd.read_csv('equip_failures_training_set.csv', na_values=['na'])

In [3]:
pd.set_option('display.max_columns',None)
mydf.head()

,id,target,sensor1_measure,sensor2_measure,sensor3_measure,sensor4_measure,sensor5_measure,sensor6_measure,sensor7_histogram_bin0,sensor7_histogram_bin1,sensor7_histogram_bin2,sensor7_histogram_bin3,sensor7_histogram_bin4,sensor7_histogram_bin5,sensor7_histogram_bin6,sensor7_histogram_bin7,sensor7_histogram_bin8,sensor7_histogram_bin9,sensor8_measure,sensor9_measure,sensor10_measure,sensor11_measure,sensor12_measure,sensor13_measure,sensor14_measure,sensor15_measure,sensor16_measure,sensor17_measure,sensor18_measure,sensor19_measure,sensor20_measure,sensor21_measure,sensor22_measure,sensor23_measure,sensor24_histogram_bin0,sensor24_histogram_bin1,sensor24_histogram_bin2,sensor24_histogram_bin3,sensor24_histogram_bin4,sensor24_histogram_bin5,sensor24_histogram_bin6,sensor24_histogram_bin7,sensor24_histogram_bin8,sensor24_histogram_bin9,sensor25_histogram_bin0,sensor25_histogram_bin1,sensor25_histogram_bin2,sensor25_histogram_bin3,sensor25_histogram_bin4,sensor25_histogram_bin5,sensor25_histogram_bin6,sensor25_histogram_bin7,sensor25_histogram_bin8,sensor25_histogram_bin9,sensor26_histogram_bin0,sensor26_histogram_bin1,sensor26_histogram_bin2,sensor26_histogram_bin3,sensor26_histogram_bin4,sensor26_histogram_bin5,sensor26_histogram_bin6,sensor26_histogram_bin7,sensor26_histogram_bin8,sensor26_histogram_bin9,sensor27_measure,sensor28_measure,sensor29_measure,sensor30_measure,sensor31_measure,sensor32_measure,sensor33_measure,sensor34_measure,sensor35_measure,sensor36_measure,sensor37_measure,sensor38_measure,sensor39_measure,sensor40_measure,sensor41_measure,sensor42_measure,sensor43_measure,sensor44_measure,sensor45_measure,sensor46_measure,sensor47_measure,sensor48_measure,sensor49_measure,sensor50_measure,sensor51_measure,sensor52_measure,sensor53_measure,sensor54_measure,sensor55_measure,sensor56_measure,sensor57_measure,sensor58_measure,sensor59_measure,sensor60_measure,sensor61_measure,sensor62_measure,sensor63_measure,sensor64_histogram_bin0,sensor64_histogram_bin1,sensor64_histogram_bin2,sensor64_histogram_bin3,sensor64_histogram_bin4,sensor64_histogram_bin5,sensor64_histogram_bin6,sensor64_histogram_bin7,sensor64_histogram_bin8,sensor64_histogram_bin9,sensor65_measure,sensor66_measure,sensor67_measure,sensor68_measure,sensor69_histogram_bin0,sensor69_histogram_bin1,sensor69_histogram_bin2,sensor69_histogram_bin3,sensor69_histogram_bin4,sensor69_histogram_bin5,sensor69_histogram_bin6,sensor69_histogram_bin7,sensor69_histogram_bin8,sensor69_histogram_bin9,sensor70_measure,sensor71_measure,sensor72_measure,sensor73_measure,sensor74_measure,sensor75_measure,sensor76_measure,sensor77_measure,sensor78_measure,sensor79_measure,sensor80_measure,sensor81_measure,sensor82_measure,sensor83_measure,sensor84_measure,sensor85_measure,sensor86_measure,sensor87_measure,sensor88_measure,sensor89_measure,sensor90_measure,sensor91_measure,sensor92_measure,sensor93_measure,sensor94_measure,sensor95_measure,sensor96_measure,sensor97_measure,sensor98_measure,sensor99_measure,sensor100_measure,sensor101_measure,sensor102_measure,sensor103_measure,sensor104_measure,sensor105_histogram_bin0,sensor105_histogram_bin1,sensor105_histogram_bin2,sensor105_histogram_bin3,sensor105_histogram_bin4,sensor105_histogram_bin5,sensor105_histogram_bin6,sensor105_histogram_bin7,sensor105_histogram_bin8,sensor105_histogram_bin9,sensor106_measure,sensor107_measure
0,1,0,76698,NaN,2.130706e+09,280.0,0.0,0.0,0.0,0.0,0.0,0.0,37250.0,1432864.0,3664156.0,1007684.0,25896.0,0.0,2551696.0,0.0,0.0,0.0,0.0,0.0,4933296.0,3655166.0,1766008.0,1132040.0,0.0,0.0,0.0,0.0,1012.0,268.0,0.0,0.0,0.0,0.0,0.0,469014.0,4239660.0,703300.0,755876.0,0.0,5374.0,2108.0,4114.0,12348.0,615248.0,5526276.0,2378.0,4.0,0.0,0.0,2328746.0,1022304.0,415432.0,287230.0,310246.0,681504.0,1118814.0,3574.0,0.0,0.0,6700214.0,0.0,10.0,108.0,50.0,2551696.0,97518.0,947550.0,799478.0,330760.0,353400.0,299160.0,305200.0,283680.0,NaN,NaN,NaN,178540.0,76698.08,6700214.0,6700214.0,6599892.0,43566.0,68656.0,54064.

In [4]:
class DropColumns(TransformerMixin):
    def __init__(self, thresh=0.15):
        self.drop_list = []
        self.thresh = thresh
        
    def fit(self, x, y=None):
        temp = (x.isnull().sum())/len(x)
        self.drop_list = [t for t in temp.index if temp[t] >= self.thresh]
        return self
    
    def transform(self, x):
        if not self.drop_list:
            return x
        return x.drop(columns=self.drop_list)

In [41]:
drop_columns = DropColumns(thresh=0.33)
imputer = SimpleImputer(strategy='median')
# imputer = IterativeImputer()
scaler = StandardScaler()

In [42]:
x, y = mydf.drop(columns=['target', 'id']), mydf['target']
mydf1 = drop_columns.fit_transform(x)
x = imputer.fit_transform(mydf1)
x = scaler.fit_transform(x)
X = pd.DataFrame(x, columns=mydf1.columns)

In [43]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

In [44]:
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 0
}

# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':500,
    #'max_features': 0.5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}

# AdaBoost parameters
ada_params = {
    'n_estimators': 500,
    'learning_rate' : 0.75
}

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 500,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}

# Support Vector Classifier parameters 
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025
    }

In [45]:
model_lgb = LGBMClassifier(learning_rat=0.1, n_estimators=400)
model_lgb.fit(X, y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rat=0.1, learning_rate=0.1,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=400, n_jobs=-1, num_leaves=31,
               objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

In [46]:
feat_imp = model_lgb.feature_importances_
feat_imp_dic = {c:feat_imp[i] for i, c in enumerate(X.columns)}

In [47]:
new_X = X[[k for k in feat_imp_dic if feat_imp_dic[k] >= 20]]

In [48]:
new_X

,sensor1_measure,sensor3_measure,sensor4_measure,sensor7_histogram_bin1,sensor7_histogram_bin2,sensor7_histogram_bin3,sensor7_histogram_bin4,sensor7_histogram_bin5,sensor7_histogram_bin6,sensor7_histogram_bin7,sensor7_histogram_bin8,sensor7_histogram_bin9,sensor8_measure,sensor9_measure,sensor10_measure,sensor12_measure,sensor13_measure,sensor14_measure,sensor15_measure,sensor16_measure,sensor17_measure,sensor18_measure,sensor20_measure,sensor22_measure,sensor23_measure,sensor24_histogram_bin0,sensor24_histogram_bin3,sensor24_histogram_bin4,sensor24_histogram_bin5,sensor24_histogram_bin6,sensor24_histogram_bin7,sensor24_histogram_bin8,sensor24_histogram_bin9,sensor25_histogram_bin0,sensor25_histogram_bin1,sensor25_histogram_bin2,sensor25_histogram_bin3,sensor25_histogram_bin4,sensor25_histogram_bin5,sensor25_histogram_bin6,sensor25_histogram_bin7,sensor25_histogram_bin8,sensor25_histogram_bin9,sensor26_histogram_bin0,sensor26_histogram_bin1,sensor26_histogram_bin2,sensor26_histogram_bin3,sensor26_histogram_bin4,sensor26_histogram_bin5,sensor26_histogram_bin6,sensor26_histogram_bin7,sensor26_histogram_bin8,sensor26_histogram_bin9,sensor27_measure,sensor28_measure,sensor29_measure,sensor30_measure,sensor31_measure,sensor33_measure,sensor34_measure,sensor35_measure,sensor44_measure,sensor45_measure,sensor46_measure,sensor48_measure,sensor49_measure,sensor50_measure,sensor51_measure,sensor52_measure,sensor53_measure,sensor55_measure,sensor56_measure,sensor57_measure,sensor59_measure,sensor60_measure,sensor61_measure,sensor62_measure,sensor63_measure,sensor64_histogram_bin0,sensor64_histogram_bin1,sensor64_histogram_bin2,sensor64_histogram_bin3,sensor64_histogram_bin4,sensor64_histogram_bin5,sensor64_histogram_bin6,sensor64_histogram_bin7,sensor64_histogram_bin8,sensor64_histogram_bin9,sensor65_measure,sensor66_measure,sensor69_histogram_bin0,sensor69_histogram_bin1,sensor69_histogram_bin2,sensor69_histogram_bin3,sensor69_histogram_bin4,sensor69_histogram_bin5,sensor69_histogram_bin6,sensor69_histogram_bin7,sensor69_histogram_bin8,sensor70_measure,sensor71_measure,sensor72_measure,sensor73_measure,sensor74_measure,sensor75_measure,sensor77_measure,sensor79_measure,sensor80_measure,sensor82_measure,sensor84_measure,sensor89_measure,sensor90_measure,sensor91_measure,sensor92_measure,sensor94_measure,sensor95_measure,sensor96_measure,sensor97_measure,sensor98_measure,sensor99_measure,sensor102_measure,sensor103_measure,sensor104_measure,sensor105_histogram_bin0,sensor105_histogram_bin1,sensor105_histogram_bin2,sensor105_histogram_bin3,sensor105_histogram_bin4,sensor105_histogram_bin5,sensor105_histogram_bin6,sensor105_histogram_bin7,sensor105_histogram_bin8,sensor105_histogram_bin9
0,0.119381,2.310224,-0.004085,-0.02837,-0.056929,-0.115643,-0.167274,0.103186,0.518102,0.362192,-0.042459,-0.029978,0.180223,-0.054934,-0.022591,-0.109015,-0.109228,0.192095,0.098135,0.250203,0.550907,-0.088073,-0.042180,-0.009916,-0.064343,-0.026718,-0.034801,-0.029628,0.258355,0.972212,-0.163286,-0.071677,-0.011802,-0.032929,-0.067166,-0.036539,-0.113842,-0.203221,0.530438,-0.109949,-0.061603,-0.044093,-0.012847,0.249553,0.057159,0.004152,0.019918,0.198765,0.974940,1.435939,-0.344766,-0.146400,-0.106043,0.202886,-0.137658,-0.184726,-0.134124,-0.040790,0.263580,0.310116,0.161681,1.172227,0.119525,0.204612,0.256893,0.401803,-0.046719,0.446813,0.637595,0.262185,1.314011,-0.004083,0.048776,0.212845,-0.090339,0.094187,-0.065408,1.053696,-0.037972,-0.088790,-0.149090,-0.184919,0.010599,0.611842,0.756989,0.086650,-0.042222,-0.023110,-0.004084,-0.040174,0.489485,0.154101,0.026156,0.152996,-0.005582,0.320927,0.303646,0.146407,-0.009102,-0.018961,-0.040464,0.733061,-0.172136,-0.027322,2.384944,0.118646,0.180406,0.461154,-0.025984,-0.084558,0.297120,0.970930,1.904942,-0.045393,0.564595,0.691140,0.958719,0.641848,-0.186078,-0.128261,-0.152539,0.370080,0.419294,0.098933,0.363023,0.693832,0.524393,0.239087,0.070072,0.008264,-0.107586,-0.143103,-0.175699
1,-0.180697,-0.

In [53]:
final_model = LGBMClassifier(learning_rat=0.1, n_estimators=400)
final_model.fit(new_X, y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rat=0.1, learning_rate=0.1,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=400, n_jobs=-1, num_leaves=31,
               objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

In [49]:
scores_lgb = cross_val_score(LGBMClassifier(learning_rat=0.1, n_estimators=400),new_X, y, cv=skf, error_score='f1' )
scores_lgb.mean()

0.9944666599911667

In [50]:
test_data = pd.read_csv('equip_failures_test_set.csv', na_values=['na'])

In [51]:
x = test_data.drop(columns=['id'])
x = drop_columns.transform(x)
x = imputer.transform(x)
x = scaler.transform(x)
X_test = pd.DataFrame(x, columns=mydf1.columns)

In [54]:
new_X_test = X_test[[k for k in feat_imp_dic if feat_imp_dic[k] >= 20]]

In [56]:
pred = final_model.predict(new_X_test)

In [58]:
len(pred)

16001

In [63]:
df=pd.DataFrame(pred, columns=['target'])

In [64]:
df['id']= test_data.id

In [65]:
df.head()

,target,id
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5


In [66]:
df.describe()

,target,id
count,16001.000000,16001.000000
mean,0.019249,8001.000000
std,0.137403,4619.235164
min,0.000000,1.000000
25%,0.000000,4001.000000
50%,0.000000,8001.000000
75%,0.000000,12001.000000
max,1.000000,16001.000000
